In [17]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [18]:
# Configure LLM
apikey = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',api_key=apikey)

In [19]:
class Chatbot_State(TypedDict):
    message : Annotated[list,add_messages]

In [20]:
def chatbot(state:Chatbot_State) -> Chatbot_State:
    return {'message':[llm.invoke(state['message'])]}

In [21]:
# LangGraph

builder = StateGraph(Chatbot_State)

builder.add_node('chatting_bot',chatbot)

builder.add_edge(START,"chatting_bot")
builder.add_edge('chatting_bot',END)

graph = builder.compile()

In [22]:
print(graph.get_graph().draw_ascii())

  +-----------+  
  | __start__ |  
  +-----------+  
        *        
        *        
        *        
+--------------+ 
| chatting_bot | 
+--------------+ 
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


In [23]:
response = graph.invoke({'message':{'role':'user','content':'Who is MS Dhoni?'}})
print(response['message'][-1].content)

MS Dhoni is a legendary Indian cricketer, widely regarded as one of the greatest captains and wicket-keeper batsmen of all time.

Here's a breakdown of who he is and why he's so significant:

**Key Highlights of his Career:**

*   **Captain Marvel:** He is most famous for his incredibly successful captaincy of the Indian national team across all three formats of the game (Test, One Day International (ODI), and Twenty20 International (T20I)). Under his leadership, India achieved unprecedented success:
    *   **2007 ICC World Twenty20:** Led India to victory in the inaugural T20 World Cup.
    *   **2011 ICC Cricket World Cup:** Captained India to their second ODI World Cup title, a historic win on home soil after a 28-year drought.
    *   **2013 ICC Champions Trophy:** Led India to win this prestigious ODI tournament.
    *   **No. 1 Test Ranking:** Guided India to the top of the ICC Test rankings for the first time.
*   **Master Wicket-keeper:** He was an exceptional wicket-keeper, k

In [ ]:
# CHATBOT

while True:
  user_input=input()
  print('User:',user_input)
  if user_input.lower() in ["quit","q"]:
    print("Good Bye")
    break
  for event in graph.stream({'message':("user",user_input)}):
    for value in event.values():
      print("Assistant:",value["message"][-1].content)

User: Hi
Assistant: Hello! How can I help you today?
User: what is the date today?
Assistant: I am a large language model, I do not have access to the current date or time.
User: ok
Assistant: Alright! What's on your mind? How can I help you today? 😊
User: whta is the capital of germany?
Assistant: The capital of Germany is **Berlin**.
User: How to say good evening in german?
Assistant: The most common way to say "good evening" in German is:

**Guten Abend**

* **Guten** means "good"
* **Abend** means "evening"

You can pronounce it roughly as: **GOO-ten AH-bent**
User: q
Good Bye
